In [1]:
#PARÁMETROS
N_GRUPOS = 10
N_PASOS = 10000 #Si se aumenta, aumenta la posibilidad de que el grupo que resulte sea más heterogeneo, pero el procesamiento demorará más
N_UMBRAL = 500 #Si se aumenta, aumenta la posibilidad de que el grupo que resulte sea más heterogeneo, pero el procesamiento demorará más
N_CANDIDATOS = 1000 #Si se aumenta, el resultado será mejor, pero el procesamiento demorará más

#0) Carga y Preprocesamiento

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('Inscripción.csv')

In [4]:
df.describe()

,¿Qué tanto conoces del tema preferido?,Edad,Número de celular
count,84.000000,84.000000,8.400000e+01
mean,3.571429,22.928571,1.535308e+09
std,0.810856,3.233059,5.378152e+09
min,2.000000,17.000000,9.000000e+00
25%,3.000000,21.000000,9.423100e+08
50%,4.000000,22.000000,9.595067e+08
75%,4.000000,24.000000,9.851514e+08
max,5.000000,41.000000,5.023038e+10


In [5]:
df.shape

(84, 36)

In [6]:
df.head()

,Marca temporal,¿Cuál de los desafíos te motiva resolver?,¿Qué tanto conoces del tema preferido?,"1. ¿Alguna vez has participado en alguna hackathon, concurso o experiencia de innovación?",2. ¿Cuál de las siguientes competencias / conocimientos dominas y te identificas?,"3. Se presentará una lista de roles. En orden, elige los 5 roles que más te representen cuando trabajas en equipo. Recuerda, primero elige la que más te identifica y, así, progresivamente.",Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,"1. ¿Alguna vez has participado en alguna hackathon, concurso o experiencia de innovación?.1",2. ¿Con cuál de las siguientes competencias / conocimientos te identificas y dominas?,"3. Se presentará una lista de roles. En orden, elige los 5 roles que más te representen cuando trabajas en equipo. Recuerda, primero elige la que más te identifica y, así, progresivamente..1",Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,"1. ¿Alguna vez has participado en alguna hackathon, concurso o experiencia de innovación?.2",2. ¿Con cuál de las siguientes competencias / conocimientos te identificas y dominas? .1,"3. Se presentará una lista de roles. En orden, elige los 5 roles que más te representen cuando trabajas en equipo. Recuerda, primero elige la que más te identifica y, así, progresivamente..2",Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Apellidos,Nombres,Edad,Correo PUCP,Número de celular,Comunidad PUCP,Especialidad,"El Open INN se realizará por 4 días (del sábado 14 al martes 17 de noviembre), empezará a las 10 a.m. y finalizará a las 5:00 p.m. ¿Estarás disponible en su totalidad?",Marca los bloques en el cual NO podrás participar. Se tomará en cuenta para la selección de participantes. [Sábado 14],Marca los bloques en el cual NO podrás participar. Se tomará en cuenta para la selección de participantes. [Domingo 15],Marca los bloques en el cual NO podrás participar. Se tomará en cuenta para la selección de participantes. [Lunes 16],Marca los bloques en el cual NO podrás participar. Se tomará en cuenta para la selección de participantes. [Martes 17]
0,2/11/2020 8:08:37,Desafío 1: ¿Cómo podemos motivar e impulsar la...,5,"Sí, he participado 1 o 2 veces",Arquitectura Bioclimática / Agricultura Urbana...,Creativo/a (ej. propongo ideas),Práctico/a (ej. convierto las ideas en acciones),Mediador/a (ej. resuelvo los conflictos),Evaluador/a (ej. controlo la calidad),Metódico/a (ej. sistematizo los procesos),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Muñoz,Eizo,23,munoz.eizo@pucp.edu.pe,959506666,Estudiante,Ciencias e Ingeniería: Ingeniería Civil,Sí,NaN,NaN,NaN,NaN
1,3/11/2020 17:12:41,Desafío 1: ¿Cómo podemos motivar e impulsar la...,3,No,Metodologías de Innovación Social / User Exper...,Práctico/a (ej. convierto las ideas en acciones),Metódico/a (ej. sistematizo los procesos),Creativo/a (ej. propongo ideas),Investigador/a (ej. me especializo en algún tema),Evaluador/a (ej. controlo la calidad),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Navarro Fuentes,Roelmer Bilzaí,32,Roditas23@gmail.com,50230380644,Otro,Ciencias e Ingeniería: Ingeniería Geológica,Sí,NaN,NaN,NaN,NaN
2,4/11/2020 10:37:55,Desafío 1: ¿Cómo podemos motivar e impulsar la...,2,No,"Psicología Social / Economía Conductual, Diseñ...",Evaluador/a (ej. controlo la calidad),Investigador/a (ej. me especializo en algún tema),Coordinador/a (ej. delego tareas),Creativo/a (ej. propongo ideas),Metódico/a (ej. sistematizo los procesos),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aguirre ingunza,Nathalia,21,n.aguirre@pucp.edu.pe,953648855,Estudiante,Arte y Diseño: Diseño Industrial,"No, tengo un cruce de horarios",NaN,NaN,NaN,AM (10:00 - 1:00)
3,5/11/2020 2:34:57,Desafío 1: ¿Cómo podemos motivar e impulsar la...,4,"Sí, he participado 1 o 2 veces","Tecnologías Ambientales y Ecológicas, Arquitec...",Creativo/a (ej. propongo ideas),Coordinador/a (ej. delego tareas),Impulsor/a (ej. fomento motivación),Coordinador/a (ej. delego tareas),Práctico/a (ej. convierto

In [7]:
columnas = ['ExpTema','ExpHack','Comp','Rol1','Rol2','Rol3','Rol4','Rol5',
            'Apellido','Nombres','Carrera','Correo PUCP','Número de celular']

In [8]:
df.rename(columns={'¿Qué tanto conoces del tema preferido?':'ExpTema',\
                   '1. ¿Alguna vez has participado en alguna hackathon, concurso o experiencia de innovación?':'ExpHack',\
                   '2. ¿Cuál de las siguientes competencias / conocimientos dominas y te identificas? ':'Comp',\
                   '3. Se presentará una lista de roles. En orden, elige los 5 roles que más te representen cuando trabajas en equipo. Recuerda, primero elige la que más te identifica y, así, progresivamente.':'Rol1',\
                   'Unnamed: 6':'Rol2',\
                   'Unnamed: 7':'Rol3',\
                   'Unnamed: 8':'Rol4',\
                   'Unnamed: 9':'Rol5',\
                   'Apellidos':'Apellido',
                   'Especialidad':'Carrera'}, inplace = True)

In [9]:
df = df[columnas]

In [10]:
df.head()

,ExpTema,ExpHack,Comp,Rol1,Rol2,Rol3,Rol4,Rol5,Apellido,Nombres,Carrera,Correo PUCP,Número de celular
0,5,"Sí, he participado 1 o 2 veces",Arquitectura Bioclimática / Agricultura Urbana...,Creativo/a (ej. propongo ideas),Práctico/a (ej. convierto las ideas en acciones),Mediador/a (ej. resuelvo los conflictos),Evaluador/a (ej. controlo la calidad),Metódico/a (ej. sistematizo los procesos),Muñoz,Eizo,Ciencias e Ingeniería: Ingeniería Civil,munoz.eizo@pucp.edu.pe,959506666
1,3,No,Metodologías de Innovación Social / User Exper...,Práctico/a (ej. convierto las ideas en acciones),Metódico/a (ej. sistematizo los procesos),Creativo/a (ej. propongo ideas),Investigador/a (ej. me especializo en algún tema),Evaluador/a (ej. controlo la calidad),Navarro Fuentes,Roelmer Bilzaí,Ciencias e Ingeniería: Ingeniería Geológica,Roditas23@gmail.com,50230380644
2,2,No,"Psicología Social / Economía Conductual, Diseñ...",Evaluador/a (ej. controlo la calidad),Investigador/a (ej. me especializo en algún tema),Coordinador/a (ej. delego tareas),Creativo/a (ej. propongo ideas),Metódico/a (ej. sistematizo los procesos),Aguirre ingunza,Nathalia,Arte y Diseño: Diseño Industrial,n.aguirre@pucp.edu.pe,953648855
3,4,"Sí, he participado 1 o 2 veces","Tecnologías Ambientales y Ecológicas, Arquitec...",Creativo/a (ej. propongo ideas),Coordinador/a (ej. delego tareas),Impulsor/a (ej. fomento motivación),Coordinador/a (ej. delego tareas),Práctico/a (ej. convierto las ideas en acciones),Munoz,Eizo,Ciencias e Ingeniería: Ingeniería Civil,munoz.eizo@pucp.edu.pe,959506666
4,3,No,Gestión / Ingeniería Industrial / Marketing &...,Mediador/a (ej. resuelvo los conflictos),Evaluador/a (ej. controlo la calidad),Investigador/a (ej. me especializo en algún tema),Creativo/a (ej. propongo ideas),Metódico/a (ej. sistematizo los procesos),Romero Romero,Niel,Gestión y Alta Dirección,niel.romero@pucp.edu.pe,993146653


In [11]:
df = df.applymap(lambda x:  x.strip() if (isinstance(x, str)) else x)

#1) mini EDA

In [12]:
df['ExpTema'].value_counts()

4    38
3    29
5     9
2     8
Name: ExpTema, dtype: int64

In [13]:
df['ExpHack'].value_counts()

No                                       44
Sí, he participado 1 o 2 veces           35
Sí, he participado en varias de ellas     5
Name: ExpHack, dtype: int64

In [14]:
COL_ROLES = ['Rol1','Rol2','Rol3','Rol4','Rol5']
df[COL_ROLES] = df[COL_ROLES].applymap(lambda x: x.split()[0])

In [15]:
df['Comp'].value_counts()

Gestión / Ingeniería Industrial /  Marketing & Gestión Comercial                                                                                                                                                                                                                                                 8
Tecnología e Ingeniería (Software - Informática, Electrónica, Mecatrónica, etc.)                                                                                                                                                                                                                                 4
Tecnología e Ingeniería (Software - Informática, Electrónica, Mecatrónica, etc.), Diseño de Prototipos (Render, Figma, Marvel App, Ai, Ps etc.) / Creación de Páginas Web                                                                                                                                        3
Diseño de Prototipos (Render, modelado 3D Sketchup, Archicad, Figma, Ai, Ps etc

In [16]:
df['Carrera'].value_counts()

Ciencias e Ingeniería: Ingeniería Industrial                            16
Gestión y Alta Dirección                                                15
Ciencias e Ingeniería: Ingeniería Mecatrónica                            9
Arte y Diseño: Diseño Industrial                                         8
Ciencias e Ingeniería: Ingeniería Civil                                  7
Psicología                                                               6
Ciencias Sociales: Economía                                              3
Arquitectura y Urbanismo                                                 3
Educación                                                                3
Ciencias e Ingeniería: Ingeniería Electrónica                            3
Estudios Generales Letras                                                2
Ciencias e Ingeniería: Ingeniería Biomédica                              2
Arte y Diseño: Pintura                                                   1
Ciencias y Artes de la Co

In [17]:
df.columns

Index(['ExpTema', 'ExpHack', 'Comp', 'Rol1', 'Rol2', 'Rol3', 'Rol4', 'Rol5',
       'Apellido', 'Nombres', 'Carrera', 'Correo PUCP', 'Número de celular'],
      dtype='object')

#Preprocesar Competencias

In [18]:
#To-do
#Identificar las competencias posibles
#Para cada persona, crear una lista de competencias
test = df['Comp'].map(lambda x: x.replace(" ", "").split(','))

In [19]:
set([i for l in test for i in l ])

{'Ai',
 'Archicad',
 'ArquitecturaBioclimática/AgriculturaUrbana',
 'Arte',
 'ComunicaciónEfectiva/RelacionesPúblicas',
 'DerechosHumanosyMedioAmbiente',
 'Desarrollosostenible',
 'DiseñodePrototipos(Render',
 'Diseñodeproductos/Diseñodeservicios',
 'Diseñodeproductos/diseñodeservicios',
 'Electrónica',
 'Figma',
 'Gestión/IngenieríaIndustrial/Marketing&GestiónComercial',
 'Gestióndeproyectosyprocesos',
 'Gestiónderecursosnaturalesdentrodelhogar',
 'Gestiónsocialyempresarial',
 'Género',
 'GéneroyMasculinidades',
 'MarvelApp',
 'Mecatrónica',
 'MetodologíasdeInnovaciónSocial/UserExperience/Diseñocentradoenelusuario',
 'Metodologíasdeinnovaciónsocial/UserExperience/Diseñocentradoenusuario',
 'Psetc.)',
 'Psetc.)/CreacióndePáginasWeb',
 'PsicologíaEducacional',
 'PsicologíaSocial/EconomíaConductual',
 'PsicologíaSocial/EconomíaConductual/CienciasdelaSalud',
 'Psicologíaeducacional',
 'Sketchup',
 'Sociología/Antropología',
 'SostenibilidadAmbientaleInnovación',
 'TecnologíaeIngeniería(So

In [20]:
df['CompList'] = test

In [21]:
df.head()

,ExpTema,ExpHack,Comp,Rol1,Rol2,Rol3,Rol4,Rol5,Apellido,Nombres,Carrera,Correo PUCP,Número de celular,CompList
0,5,"Sí, he participado 1 o 2 veces",Arquitectura Bioclimática / Agricultura Urbana...,Creativo/a,Práctico/a,Mediador/a,Evaluador/a,Metódico/a,Muñoz,Eizo,Ciencias e Ingeniería: Ingeniería Civil,munoz.eizo@pucp.edu.pe,959506666,"[ArquitecturaBioclimática/AgriculturaUrbana, G..."
1,3,No,Metodologías de Innovación Social / User Exper...,Práctico/a,Metódico/a,Creativo/a,Investigador/a,Evaluador/a,Navarro Fuentes,Roelmer Bilzaí,Ciencias e Ingeniería: Ingeniería Geológica,Roditas23@gmail.com,50230380644,[MetodologíasdeInnovaciónSocial/UserExperience...
2,2,No,"Psicología Social / Economía Conductual, Diseñ...",Evaluador/a,Investigador/a,Coordinador/a,Creativo/a,Metódico/a,Aguirre ingunza,Nathalia,Arte y Diseño: Diseño Industrial,n.aguirre@pucp.edu.pe,953648855,"[PsicologíaSocial/EconomíaConductual, Diseñode..."
3,4,"Sí, he participado 1 o 2 veces","Tecnologías Ambientales y Ecológicas, Arquitec...",Creativo/a,Coordinador/a,Impulsor/a,Coordinador/a,Práctico/a,Munoz,Eizo,Ciencias e Ingeniería: Ingeniería Civil,munoz.eizo@pucp.edu.pe,959506666,"[TecnologíasAmbientalesyEcológicas, Arquitectu..."
4,3,No,Gestión / Ingeniería Industrial / Marketing &...,Mediador/a,Evaluador/a,Investigador/a,Creativo/a,Metódico/a,Romero Romero,Niel,Gestión y Alta Dirección,niel.romero@pucp.edu.pe,993146653,[Gestión/IngenieríaIndustrial/Marketing&Gestió...


#Preprocessar Carreras

In [22]:
#To-do
#Identificar las carreras posibles
#Hacer reemplazos equivalentes (arquitecto = arquitectura ~= arq y urb) Hardcodeo aquí
#Para cada persona, crear una lista de competencias

#Preprocesar roles

Los roles convierten a cada persona en un vector con las siguientes dimensiones ['Práctico/a',
 'Creativo/a',
 'Coordinador/a',
 'Mediador/a',
 'Metódico/a',
 'Impulsor/a',
 'Investigador/a',
 'Evaluador/a']. Si yo marqué que mi rol1 es Práctico, mi rol2 es Metódico, rol3 Mediador, rol4 Investigador y rol5 Coordinador; mi vector será [16,1,4,8,0,2,0].

In [23]:
COL_ROLES = ['Rol1','Rol2','Rol3','Rol4','Rol5']
roles = pd.concat([df[r] for r in COL_ROLES]).value_counts()
ROLES = list(pd.Series(roles.index).map(lambda x: x.split()[0]).values)
ROLES

['Creativo/a',
 'Coordinador/a',
 'Práctico/a',
 'Investigador/a',
 'Evaluador/a',
 'Mediador/a',
 'Impulsor/a',
 'Metódico/a']

In [24]:
import numpy as np

def rank_rol(row):
  d = dict(zip(ROLES,[0]*len(ROLES)))

  peso = 16
  for col in COL_ROLES:
    d[row[col]] = peso 
    peso = peso / 2

  return np.array(list(d.values()))

In [25]:
vectores_roles = df.apply(rank_rol, axis=1)

#Crear personas

In [26]:
class persona:
  def __init__(self,pesos,carrera,competencia,exp_tema,exp_hack,apellido,nombres,correo,tlf):
    self.pesos = pesos
    self.carrera = carrera
    self.competencia = competencia
    self.exp_tema = exp_tema
    self.apellido = apellido
    self.nombres = nombres
    self.correo = correo
    self.tlf = tlf

    if (exp_hack == 'Sí, he participado 1 o 2 veces'): self.exp_hack = 2
    elif (exp_hack == 'Sí, he participado en varias de ellas'): self.exp_hack = 5
    else: self.exp_hack = 1


In [27]:
def crear_persona(row):
  pesos = rank_rol(row)

  return persona(pesos,row['Carrera'],row['CompList'],row['ExpTema'],row['ExpHack'],row['Apellido'],row['Nombres'],row['Correo PUCP'],row['Número de celular'])

In [28]:
personas = list(df.apply(crear_persona, axis=1).values)

In [29]:
df.head()

,ExpTema,ExpHack,Comp,Rol1,Rol2,Rol3,Rol4,Rol5,Apellido,Nombres,Carrera,Correo PUCP,Número de celular,CompList
0,5,"Sí, he participado 1 o 2 veces",Arquitectura Bioclimática / Agricultura Urbana...,Creativo/a,Práctico/a,Mediador/a,Evaluador/a,Metódico/a,Muñoz,Eizo,Ciencias e Ingeniería: Ingeniería Civil,munoz.eizo@pucp.edu.pe,959506666,"[ArquitecturaBioclimática/AgriculturaUrbana, G..."
1,3,No,Metodologías de Innovación Social / User Exper...,Práctico/a,Metódico/a,Creativo/a,Investigador/a,Evaluador/a,Navarro Fuentes,Roelmer Bilzaí,Ciencias e Ingeniería: Ingeniería Geológica,Roditas23@gmail.com,50230380644,[MetodologíasdeInnovaciónSocial/UserExperience...
2,2,No,"Psicología Social / Economía Conductual, Diseñ...",Evaluador/a,Investigador/a,Coordinador/a,Creativo/a,Metódico/a,Aguirre ingunza,Nathalia,Arte y Diseño: Diseño Industrial,n.aguirre@pucp.edu.pe,953648855,"[PsicologíaSocial/EconomíaConductual, Diseñode..."
3,4,"Sí, he participado 1 o 2 veces","Tecnologías Ambientales y Ecológicas, Arquitec...",Creativo/a,Coordinador/a,Impulsor/a,Coordinador/a,Práctico/a,Munoz,Eizo,Ciencias e Ingeniería: Ingeniería Civil,munoz.eizo@pucp.edu.pe,959506666,"[TecnologíasAmbientalesyEcológicas, Arquitectu..."
4,3,No,Gestión / Ingeniería Industrial / Marketing &...,Mediador/a,Evaluador/a,Investigador/a,Creativo/a,Metódico/a,Romero Romero,Niel,Gestión y Alta Dirección,niel.romero@pucp.edu.pe,993146653,[Gestión/IngenieríaIndustrial/Marketing&Gestió...


#Grupos

In [30]:
#Cálcular número de integrantes en cada grupo

In [31]:
n_personas = len(personas)

In [32]:
min_personas = n_personas // N_GRUPOS
n_grupos_uno_extra = n_personas - min_personas * N_GRUPOS

In [33]:
print('Para',N_GRUPOS,'grupos se crearán',N_GRUPOS-n_grupos_uno_extra,'grupos de'
,min_personas,'personas y',n_grupos_uno_extra,'de',min_personas+1,'personas')

Para 10 grupos se crearán 6 grupos de 8 personas y 4 de 9 personas


In [34]:
comb_cero = [personas[x*min_personas:(x+1)*min_personas] for x in range(N_GRUPOS-n_grupos_uno_extra)] + \
[personas[(N_GRUPOS-n_grupos_uno_extra)*min_personas + x*(min_personas+1):(N_GRUPOS-n_grupos_uno_extra)*min_personas + (x+1)*(min_personas+1)] for x in range(n_grupos_uno_extra)]

In [35]:
df.Apellido.values

array(['Muñoz', 'Navarro Fuentes', 'Aguirre ingunza', 'Munoz',
       'Romero Romero', 'Gamboa Rojas', 'Chumacero Ponte',
       'Significación Mejia', 'Pacahuala cubas', 'Noborikawa Gushiken',
       'Leivo San', 'Paima', 'Inga Moncada', 'Medina Herrera',
       'Zuñiga Linares', 'Mardini Aranda', 'Hurtado Roca',
       'Pachas Pachas', 'Verioska', 'Aguilar Clavijo', 'COTERA PUMACAYO',
       'Goicochea Serrano', 'Farfan Lopez', 'Cueva Vásquez',
       'Ojeda Cipriani', 'Bautista Agurto', 'Garcia Murguia',
       'Pajuelo seminario', 'TAVARA DAVILA', 'Yalle De la Cruz',
       'Rojas Sotelo', 'Gómez Aliaga', 'Luján Sandoval',
       'Yanqui Magallan', 'Ttito Chavez', 'ROMERO YSLACHIN',
       'Echevarria Concha', 'Hernani Valderrama', 'Dávila Tirado',
       'Ortega Garcia', 'Alfaro Guerrero', 'Andia Alva',
       'Saenz de la Cruz', 'Castillo Huashuayo', 'Contreras Paucca',
       'Luján Sandoval', 'Valdivia Quiñones', 'CRUZ PADILLA',
       'Tejeda Montero', 'Dionicio Arzapalo', 'Mo

In [36]:
for i,grupo in enumerate(comb_cero):
  print('Grupo',str(i),':')
  for persona in grupo:
    print(persona.apellido)

Grupo 0 :
Muñoz
Navarro Fuentes
Aguirre ingunza
Munoz
Romero Romero
Gamboa Rojas
Chumacero Ponte
Significación Mejia
Grupo 1 :
Pacahuala cubas
Noborikawa Gushiken
Leivo San
Paima
Inga Moncada
Medina Herrera
Zuñiga Linares
Mardini Aranda
Grupo 2 :
Hurtado Roca
Pachas Pachas
Verioska
Aguilar Clavijo
COTERA PUMACAYO
Goicochea Serrano
Farfan Lopez
Cueva Vásquez
Grupo 3 :
Ojeda Cipriani
Bautista Agurto
Garcia Murguia
Pajuelo seminario
TAVARA DAVILA
Yalle De la Cruz
Rojas Sotelo
Gómez Aliaga
Grupo 4 :
Luján Sandoval
Yanqui Magallan
Ttito Chavez
ROMERO YSLACHIN
Echevarria Concha
Hernani Valderrama
Dávila Tirado
Ortega Garcia
Grupo 5 :
Alfaro Guerrero
Andia Alva
Saenz de la Cruz
Castillo Huashuayo
Contreras Paucca
Luján Sandoval
Valdivia Quiñones
CRUZ PADILLA
Grupo 6 :
Tejeda Montero
Dionicio Arzapalo
Morón Salazar
Angulo Álvarez
Perez
Peinado Bravo
Tamura Tawata
Peinado Bravo
Muñoz Arista
Grupo 7 :
Pacheco Chavarría
Robles Casas
Gonzales Cavel
Reynel Joyja
Núñez Rodas
Diaz Zevallos
Olano Valo

#Buscando grupos heterogéneos

In [37]:
import random
def random_swap(comb):
  listas = random.sample(comb,2)
  l1 = listas[0]
  l2 = listas[1]
  i1 = random.randint(0,len(l1)-1)
  i2 = random.randint(0,len(l2)-1)
  aux = l1[i1]
  l1[i1] = l2[i2]
  l2[i2] = aux
  return comb

In [38]:
def calcular_costo_competencia(comb):
  suma = 0
  for grupo in comb:
    com_count = np.array([c for p in grupo for c in p.competencia])
    suma += sum(np.unique(com_count, return_counts=True)[1] - 1)
  return suma

In [39]:
import copy
def calcular_combinacion_candidata():
  comb = copy.deepcopy(comb_cero)
  min_costo = 10000
  best_comb = None
  cont = 0 #Sirve para salir si no se supera el umbral tras N_UMBRAL iteraciones
  for _ in range(N_PASOS):
    comb = random_swap(comb)
    costo = calcular_costo_competencia(comb)
    if (costo < min_costo):
      best_comb = copy.deepcopy(comb)
      min_costo = costo
      cont = 0
    cont += 1
    if (cont > N_UMBRAL): break
  
  return best_comb

In [40]:
best_comb = calcular_combinacion_candidata()

In [41]:
calcular_costo_competencia(best_comb)

151

In [42]:
for i,grupo in enumerate(best_comb):
  print('Grupo',str(i),':')
  for persona in grupo:
    print(persona.apellido,persona.competencia)

Grupo 0 :
Yurivilca Canchán ['TecnologíaeIngeniería(Software-Informática', 'Electrónica', 'Mecatrónica', 'etc.)']
Contreras Paucca ['TecnologíaeIngeniería(Software-Informática', 'Electrónica', 'Mecatrónica', 'etc.)', 'DiseñodePrototipos(Render', 'Figma', 'MarvelApp', 'Ai', 'Psetc.)/CreacióndePáginasWeb']
Pachas Pachas ['TecnologíasAmbientalesyEcológicas', 'Gestiónderecursosnaturalesdentrodelhogar', 'MetodologíasdeInnovaciónSocial/UserExperience/Diseñocentradoenelusuario']
Aguilar Clavijo ['TecnologíasAmbientalesyEcológicas', 'Gestión/IngenieríaIndustrial/Marketing&GestiónComercial']
Tamura Tawata ['DiseñodePrototipos(Render', 'modelado3DSketchup', 'Archicad', 'Figma', 'Ai', 'Psetc.)', 'Diseñodeproductos/diseñodeservicios', 'Metodologíasdeinnovaciónsocial/UserExperience/Diseñocentradoenusuario']
Echevarria Concha ['TecnologíasAmbientalesyEcológicas', 'TecnologíaeIngeniería(Software-Informática', 'Electrónica', 'Mecatrónica', 'etc.)', 'Diseñodeproductos/Diseñodeservicios', 'Metodologíasd

In [43]:
#Consiguiendo conjunto de N_CANDIDATOS de combinaciones
comb_candidatas = []
while (len(comb_candidatas)<N_CANDIDATOS):
  cand = calcular_combinacion_candidata()
  comb_candidatas.append(cand)

#Eligiendo combinación más balanceada

In [44]:
#calcular balance de la combinación
import itertools
def get_distancia_baricentros(comb):
  baricentros = []
  for i,grupo in enumerate(comb):
    suma_pesos = 0
    for persona in grupo:
      m1 = np.multiply(persona.pesos, persona.exp_tema)
      m2 = np.multiply(m1, persona.exp_hack)
      suma_pesos +=  m2
    baricentros.append(suma_pesos/len(grupo))

  max_distance = 0
  for a,b in itertools.combinations(baricentros, 2):
    distance = np.linalg.norm(a-b)
    if ( distance >  max_distance): max_distance = distance

  return max_distance

In [45]:
min_distance = 99999

for comb in comb_candidatas:
  distance = get_distancia_baricentros(comb)
  if (distance < min_distance):
    best_comb = comb
    min_distance = distance

print(min_distance)
print(calcular_costo_competencia(best_comb))

54.87941326216963
143


In [46]:
for i,grupo in enumerate(best_comb):
  print('Grupo',str(i),':')
  for persona in grupo:
    print(persona.apellido,persona.competencia)

Grupo 0 :
Terranova veliz ['TecnologíasEmergentes/TecnologíaSocial', 'DiseñodePrototipos(Render', 'modelado3D', 'Sketchup', 'Archicad', 'Figma', 'Ai', 'Psetc.)', 'Diseñodeproductos/Diseñodeservicios', 'Metodologíasdeinnovaciónsocial/UserExperience/Diseñocentradoenusuario']
Ojeda Cipriani ['Gestión/IngenieríaIndustrial/Marketing&GestiónComercial']
Ttito Chavez ['TecnologíasAmbientalesyEcológicas', 'TecnologíaeIngeniería(Software-Informática', 'Electrónica', 'Mecatrónica', 'etc.)', 'Diseñodeproductos/Diseñodeservicios']
Yurivilca Canchán ['TecnologíaeIngeniería(Software-Informática', 'Electrónica', 'Mecatrónica', 'etc.)']
Mardini Aranda ['TecnologíasAmbientalesyEcológicas', 'ArquitecturaBioclimática/AgriculturaUrbana']
Rojas Sotelo ['TecnologíasAmbientalesyEcológicas', 'Gestión/IngenieríaIndustrial/Marketing&GestiónComercial', 'Diseñodeproductos/Diseñodeservicios']
Urbe Mostajo ['TecnologíasEmergentes/TecnologíaSocial', 'SostenibilidadAmbientaleInnovación', 'Género', 'patronesdegéneroyre

#Grabando en tabla

In [47]:
data =[]
for i,grupo in enumerate(best_comb):
  for persona in grupo:
    row = ['Grupo'+str(i+1),persona.apellido,persona.nombres,persona.competencia,persona.correo,persona.tlf]
    data.append(row)

In [48]:
resultado = pd.DataFrame(data)

In [49]:
resultado.columns = ['GRUPO','NOMBRES','APELLIDOS','COMPETENCIAS','CORREO','TELÉFONO']

In [50]:
resultado.to_excel('Grupos.xlsx',sheet_name='GRUPOS')